# Исследование алгоритма BoVW для нашей задачи

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
from dataset.get_pictures import DatasetOperations
DatasetOperations.clear()

In [2]:
PATH = f"results"

In [3]:
links = [link for link in os.listdir(PATH)]
links

['GradientBoosting_KMeans_AKAZE.json',
 'GradientBoosting_KMeans_ORB.json',
 'GradientBoosting_KMeans_Resnet.json',
 'GradientBoosting_KMeans_SIFT.json',
 'GradientBoosting_MiniBatchKMeans_AKAZE.json',
 'GradientBoosting_MiniBatchKMeans_ORB.json',
 'GradientBoosting_MiniBatchKMeans_Resnet.json',
 'GradientBoosting_MiniBatchKMeans_SIFT.json',
 'LogisticRegression_KMeans_AKAZE.json',
 'LogisticRegression_KMeans_ORB.json',
 'LogisticRegression_KMeans_Resnet.json',
 'LogisticRegression_KMeans_SIFT.json',
 'LogisticRegression_MiniBatchKMeans_AKAZE.json',
 'LogisticRegression_MiniBatchKMeans_ORB.json',
 'LogisticRegression_MiniBatchKMeans_Resnet.json',
 'LogisticRegression_MiniBatchKMeans_SIFT.json',
 'SVM_KMeans_AKAZE.json',
 'SVM_KMeans_ORB.json',
 'SVM_KMeans_Resnet.json',
 'SVM_KMeans_SIFT.json',
 'SVM_MiniBatchKMeans_AKAZE.json',
 'SVM_MiniBatchKMeans_ORB.json',
 'SVM_MiniBatchKMeans_Resnet.json',
 'SVM_MiniBatchKMeans_SIFT.json']

In [10]:
k = 0
for link in links:
    with open(f"{PATH}\\{link}", "r") as file:
        data = json.load(file)
        if data['testing']['accurancy'] == 1.0:
            print(link)
            k += 1
            
k

LogisticRegression_KMeans_AKAZE.json
LogisticRegression_KMeans_ORB.json
LogisticRegression_KMeans_SIFT.json
LogisticRegression_MiniBatchKMeans_ORB.json
SVM_KMeans_ORB.json
SVM_KMeans_SIFT.json
SVM_MiniBatchKMeans_ORB.json


7

In [7]:
k = 0
for link in links:
    with open(f"{PATH}\\{link}", "r") as file:
        data = json.load(file)
        if data['result'][0] == "artist":
            print(f"{link}: {data['testing']['accurancy']}, {data['result'][1]}, {data['result'][2]}")
            k += 1
            
k

LogisticRegression_KMeans_AKAZE.json: 1.0, other_artist, other_artist
LogisticRegression_KMeans_ORB.json: 1.0, other_artist, other_artist
LogisticRegression_KMeans_Resnet.json: 0.25, artist, artist
LogisticRegression_KMeans_SIFT.json: 1.0, other_artist, other_artist
LogisticRegression_MiniBatchKMeans_AKAZE.json: 0.875, other_artist, other_artist
LogisticRegression_MiniBatchKMeans_ORB.json: 1.0, other_artist, other_artist
LogisticRegression_MiniBatchKMeans_Resnet.json: 0.875, other_artist, other_artist
LogisticRegression_MiniBatchKMeans_SIFT.json: 0.875, other_artist, other_artist
SVM_KMeans_AKAZE.json: 0.875, other_artist, other_artist
SVM_KMeans_ORB.json: 1.0, other_artist, other_artist
SVM_KMeans_Resnet.json: 0.25, artist, artist
SVM_KMeans_SIFT.json: 1.0, other_artist, other_artist
SVM_MiniBatchKMeans_AKAZE.json: 0.875, other_artist, other_artist
SVM_MiniBatchKMeans_ORB.json: 1.0, other_artist, other_artist
SVM_MiniBatchKMeans_Resnet.json: 0.375, artist, artist
SVM_MiniBatchKMeans_S

16